In [1]:
import pandas as pd

In [2]:
links = pd.read_csv('wbsflix-dataset/ml-latest-small/links.csv')
movies = pd.read_csv('wbsflix-dataset/ml-latest-small/movies.csv')
ratings = pd.read_csv('wbsflix-dataset/ml-latest-small/ratings.csv')
tags = pd.read_csv('wbsflix-dataset/ml-latest-small/tags.csv')

In [3]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


Function that takes as an input a user id and outputs the top n movies:

In [9]:
top_movies_for_id = []
def top_movies(df_rate, df_movies, name, n):
    df_merge = df_rate.merge(df_movies, how='inner', on='movieId').filter(['userId', 'rating', 'title'])
    df_pivot = pd.pivot_table(df_merge, values='rating', columns='title', index='userId')
    df_movie_list = df_pivot.columns.values.tolist()
    for value in df_movie_list:
        if df_pivot.loc[name][value] == 5:
            top_movies_for_id.append(value)
            if len(top_movies_for_id) == n:
                break
    return top_movies_for_id

In [10]:
top_movies(ratings, movies, 1, 30)

['Adventures of Robin Hood, The (1938)',
 'Alice in Wonderland (1951)',
 'All Quiet on the Western Front (1930)',
 'American Beauty (1999)',
 'American History X (1998)',
 'American Tail, An (1986)',
 'Austin Powers: International Man of Mystery (1997)',
 'Back to the Future (1985)',
 'Bambi (1942)',
 'Basic Instinct (1992)',
 'Bedknobs and Broomsticks (1971)',
 'Big Lebowski, The (1998)',
 'Billy Madison (1995)',
 'Black Cauldron, The (1985)',
 'Blazing Saddles (1974)',
 'Blues Brothers, The (1980)',
 'Bottle Rocket (1996)',
 'Canadian Bacon (1995)',
 "Charlotte's Web (1973)",
 'Citizen Kane (1941)',
 'Clockwork Orange, A (1971)',
 'Conan the Barbarian (1982)',
 'Crocodile Dundee (1986)',
 'Desperado (1995)',
 'Dirty Dozen, The (1967)',
 'Dogma (1999)',
 'Dr. No (1962)',
 'Duck Soup (1933)',
 'Dumb & Dumber (Dumb and Dumber) (1994)',
 'Dumbo (1941)']

Function to get the most popular movies

In [6]:
def pop_movies(df_rate, df_movies, rate_tresh):
    df_merge = df_rate.merge(df_movies, how='inner', on='movieId').filter(['userId', 'rating', 'title'])
    df = df_merge.groupby('title').agg(rate_count=('rating','count'), rate_mean=('rating','mean')).query('rate_count >= @rate_tresh').sort_values('rate_mean', ascending=False)
    return df.index.to_list()

In [7]:
pop_movies(ratings, movies, 200)

['Shawshank Redemption, The (1994)',
 'Fight Club (1999)',
 'Usual Suspects, The (1995)',
 'Star Wars: Episode IV - A New Hope (1977)',
 "Schindler's List (1993)",
 'Star Wars: Episode V - The Empire Strikes Back (1980)',
 'Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)',
 'Pulp Fiction (1994)',
 'Matrix, The (1999)',
 'Forrest Gump (1994)',
 'Silence of the Lambs, The (1991)',
 'American Beauty (1999)',
 'Braveheart (1995)',
 'Seven (a.k.a. Se7en) (1995)',
 'Terminator 2: Judgment Day (1991)',
 'Toy Story (1995)',
 'Apollo 13 (1995)',
 'Jurassic Park (1993)',
 'Independence Day (a.k.a. ID4) (1996)']